In [2]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import pandas as pd
from IPython.display import display
from tqdm import tqdm, tqdm_notebook, trange
import sentencepiece as spm
import wget
import random
import copy
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
from torch.autograd import Variable

## Data Load

In [3]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [5]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cpu


In [6]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True, fix_length=512)
LABEL = data.Field(sequential=False, batch_first=True)

In [7]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [8]:
print('trainset 구성요소 출력 :', trainset.fields)
print("testset 구성요소 출력 :", testset.fields)

trainset 구성요소 출력 : {'text': <torchtext.data.field.Field object at 0x000001E990CA3DF0>, 'label': <torchtext.data.field.Field object at 0x000001E990D86490>}
testset 구성요소 출력 : {'text': <torchtext.data.field.Field object at 0x000001E990CA3DF0>, 'label': <torchtext.data.field.Field object at 0x000001E990D86490>}


In [9]:
print(vars(trainset[0])['text'])
print(vars(trainset[0])['label'])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', 'your', 't

In [10]:
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

In [11]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 46159
클래스의 개수 : 2


In [12]:
trainset, valset = trainset.split(split_ratio=0.8)

In [13]:
# BATCH_SIZE = 64
# [64, 512]로 고정되도록 패딩
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset), batch_size=BATCH_SIZE, device=DEVICE)

In [14]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


In [15]:
batch = next(iter(train_iter))
print(batch.text.shape)

torch.Size([64, 512])


In [16]:
batch = next(iter(train_iter))
print(batch.text.shape)

torch.Size([64, 512])


In [17]:
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset,valset,testset), batch_size=BATCH_SIZE, shuffle=True, repeat=False)

## Model Architecture

* reference    
http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [18]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [19]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [20]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [21]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [22]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        drop = sublayer(self.norm(x))
        return x + self.dropout(drop[0])

In [23]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [24]:
"""
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    x = torch.matmul(p_attn, value)
    x = x.permute(0,2,1,3).contiguous()
    x = x.view(BATCH_SIZE, -1, d_k)

    return x, p_attn
    """

'\ndef attention(query, key, value, mask=None, dropout=None):\n    "Compute \'Scaled Dot Product Attention\'"\n    d_k = query.size(-1)\n    scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k)\n    if mask is not None:\n        scores = scores.masked_fill(mask == 0, -1e9)\n    p_attn = F.softmax(scores, dim = -1)\n    if dropout is not None:\n        p_attn = dropout(p_attn)\n    \n    x = torch.matmul(p_attn, value)\n    x = x.permute(0,2,1,3).contiguous()\n    x = x.view(BATCH_SIZE, -1, d_k)\n\n    return x, p_attn\n    '

In [25]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, hid_dim, device, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert hid_dim % n_heads == 0
        # We assume d_v always equals head_dim
        self.head_dim = hid_dim // n_heads
        self.n_heads = n_heads
        self.hid_dim = hid_dim

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        #self.linears = clones(nn.Linear(hid_dim, hid_dim), 4)
        #self.attn = None
        self.dropout = nn.Dropout(p=dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask=None):
        
        batch_size = query.shape[0]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

        energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale

        if mask is not None:
            energy = energy.masked_fill(mask==0, -1e10)
        attention = torch.softmax(energy, dim=-1)

        x = torch.matmul(self.dropout(attention),V)
        x = x.permute(0,2,1,3).contiguous()
        x = x.view(batch_size, -1, self.hid_dim)
        x = self.fc_o(x)

        return x, attention


        """
        "Implements Figure 2"
        #if mask is not None:
            # Same mask applied to all h heads.
            #mask = mask.unsqueeze(1)
        query = query.transpose(0, 1)
        key = key.transpose(0, 1)
        value = value.transpose(0, 1)

        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from hid_dim => h x head_dim 
        query, key, value = \
            [l(x).view(nbatches, -1, self.n_heads, self.head_dim).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.n_heads * self.head_dim)
        return self.linears[-1](x)
        """

In [26]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [27]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [28]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [29]:
class TransformerModel(nn.Module):
    """
    """   
        
        
    def __init__(self, n_token, n_dim_model, n_head, n_hidden, n_blocks, src_pad_idx, dropout=0.5):
        super(TransformerModel, self).__init__()
        
        # #Multi Headed Attention Layer
        self_attention = MultiHeadedAttention(n_head, n_dim_model, DEVICE)
        # #Feedforward Layer
        feed_forward = PositionwiseFeedForward(n_dim_model, n_hidden, dropout)
        # #Positional Encoding
        positional_encoding = PositionalEncoding(n_dim_model, dropout)
        
        encoder_layer = EncoderLayer(n_dim_model, copy.deepcopy(self_attention), copy.deepcopy(feed_forward), dropout)
        self.encoder = Encoder(encoder_layer, n_blocks)

        embedding = Embeddings(n_dim_model, n_token)
        self.src_embed = nn.Sequential(embedding, copy.deepcopy(positional_encoding))
        
        # Fully-Connected Layer
        self.fc = nn.Linear(n_dim_model, 2)

        self.src_pad_idx = src_pad_idx


    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    

    def forward(self, x):
        
        mask = self.make_src_mask(x)

        # # x dimension[k, batch_size = 64]
        embedded_sents = self.src_embed(x)
        encoded_sents = self.encoder(embedded_sents, mask)
        
       
        final_feature_map = encoded_sents[:,-1,:] 
        final_out = self.fc(final_feature_map) 
        
        return final_out 

## Train

In [30]:
INPUT_DIM = len(TEXT.vocab)
MODEL_DIM = 512
HIDDEN_DIM = 256
N_LAYERS = 3
N_HEADS = 8
DROPOUT_RATIO = 0.1

In [31]:
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
LABEL_PAD_IDX = LABEL.vocab.stoi[LABEL.pad_token]

model = TransformerModel(INPUT_DIM, MODEL_DIM, N_HEADS, HIDDEN_DIM, N_LAYERS, TEXT_PAD_IDX, DROPOUT_RATIO)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 27,582,210 trainable parameters


In [33]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

TransformerModel(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (fc_q): Linear(in_features=512, out_features=512, bias=True)
          (fc_k): Linear(in_features=512, out_features=512, bias=True)
          (fc_v): Linear(in_features=512, out_features=512, bias=True)
          (fc_o): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=256, bias=True)
          (w_2): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, in

In [34]:
for b, batch in enumerate(train_iter):
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    y.data.sub_(1)
    print(x.size())
    print(y.size())
    optimizer.zero_grad()
    
    logit = model(x)
    print(logit)
    print(logit.size())
    
    loss = criterion(logit, y)
    print(loss)
    loss.backward()
    optimizer.step()
    break

torch.Size([64, 512])
torch.Size([64])
tensor([[ 0.2253,  1.7204],
        [-0.3326,  0.9412],
        [ 0.1029,  1.8284],
        [-0.4344,  1.7913],
        [-0.6217,  1.7387],
        [-0.2105,  1.3229],
        [-0.3129,  1.1593],
        [ 0.1973,  1.3737],
        [ 0.3140,  1.3639],
        [-0.1225,  1.2276],
        [-0.5998,  1.4094],
        [-0.2856,  1.6998],
        [-0.6404,  1.8271],
        [ 0.0615,  1.9545],
        [ 0.1687,  1.2195],
        [-0.0895,  0.8873],
        [-0.9595,  1.6694],
        [-0.3372,  1.9250],
        [ 0.2530,  1.2232],
        [-0.4604,  1.3977],
        [-0.8378,  1.3367],
        [-0.3294,  1.4709],
        [ 0.4598,  1.5131],
        [-0.0127,  1.3757],
        [ 0.1510,  1.0796],
        [ 0.3030,  0.8019],
        [-0.1440,  1.2452],
        [-0.5162,  1.6777],
        [ 0.0281,  1.7306],
        [-0.2237,  1.8167],
        [ 0.2185,  1.2288],
        [-0.6588,  1.6198],
        [-0.5282,  1.2838],
        [-0.0115,  1.3192],
        [

In [67]:
def train(model, optimizer, criterion, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()

In [68]:
def evaluate(model, criterion, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = criterion(logit, y)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [69]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, criterion, train_iter)
    val_loss, val_accuracy = evaluate(model, criterion, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), 'D:/IIPL_internship/Week1/txtclassification.pt')
        best_val_loss = val_loss

TypeError: forward() got an unexpected keyword argument 'reduction'